<a href="https://colab.research.google.com/github/hideousmaiden/youngtyp_corpora/blob/main/project_nlp_Kozlova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pdfplumber

In [ ]:
import requests
from bs4 import BeautifulSoup
import pdfplumber
import re
import os
import json

In [ ]:
session = requests.session()

In [ ]:
def intToRoman(num):  # to convert arabic numbers into roman
    rules = (("L", 50), ("XL", 40), ("XXX", 30), ("XX", 20), ("X", 10), ("IX", 9), ("V", 5), ("IV", 4), ("I", 1),)
    res = ""
    for suf, val in rules:
        while num >= val:
            num -= val
            res += suf     
    return res

In [ ]:
def download_pdf(link):  # to download the pdf file
    req = session.get(link)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    texts = soup.find_all('p', {'style': 'text-align: center;'})
    links = []
    for text in texts:
        if 'Тезисы' in text.text:
            for link in text.find_all('a'):
                links.append(link.get('href'))
    url_theses = links[1]
    res = requests.get(url_theses)
    return res.content

In [ ]:
def clear_end(txt):  # to cut the beginning and the end of the book
    if 'Издательство «Нестор-История»' in txt:
        txt = txt.split('Издательство «Нестор-История»')[1].strip(', \n')
    if 'Д о к л а д ы\n' in txt:
        txt = txt.split('Д о к л а д ы\n')[1].strip()
    if 'Доклады\n' in txt:
        txt = txt.split('Доклады\n')[1].strip()
    if '\nТезисы докладов\n' in txt:
        txt = txt.split('Тезисы докладов\n')[1].strip()
    if 'ИЛИ РАН,\n' in txt:
        txt = txt.split('ИЛИ РАН,\n')[1].strip()
    if 'Часть I\n' in txt:
        txt = txt.split('Часть I\n')[1].strip()
    return txt

In [ ]:
for i in range(4, 19):  # downloading PDF-files
    if i == 16 or i == 8:
        continue
    if not os.path.exists(f'theses_{2003 + i}/'):  # creating a folder for each year
        os.mkdir(f'theses_{2003 + i}/')
    if i == 18:
        url = f'https://www.youngconfspb.com/proshedshie-konferencii/{intToRoman(i).lower()}-konferenciya-{2003 + i}-g'
    else:
        url = f'https://www.youngconfspb.com/{intToRoman(i).lower()}-konferenciya-{2003 + i}-g'
    response = download_pdf(url)
    with open(f'theses_{2003 + i}/theses_{2003 + i}.pdf', "wb") as f:
        f.write(response)

In [ ]:
for j in range(2007, 2022):
    if j == 2019 or j == 2011:
        continue
    end_text = ''
    with pdfplumber.open(f'theses_{j}/theses_{j}.pdf') as pdf:  # opening PDF-file as TXT
        for p in range(len(pdf.pages)):  # clearing the text
            page = pdf.pages[p]
            text = page.extract_text()
            text = text.strip().strip('1234567890').strip()
            text = text.split('\n')
            for part in text:
                part = part.strip()
                if part.endswith('-'):
                    part = part.rstrip('-')
                    end_text += part
                else:
                    end_text += '\n' + part
                end_text = end_text.strip()
    if j == 2010:  # cutting ends again
        end_text = 'СОДЕРЖАНИЕ'.join(end_text.split('СОДЕРЖАНИЕ')[:-1])
    elif j == 2012:
        end_text = end_text.split('СОДЕРЖАНИЕ')[1].split('.......................................')[-1].strip('0123456789').strip()
    else:                                
        end_text = end_text.split('ОГЛАВЛЕНИЕ')[0].split('СОДЕРЖАНИЕ')[0]
    with open(f'theses_{j}/theses_{j}.txt', 'w', encoding='utf-8') as file:  # writing texts in the file
        file.write(clear_end(end_text))

In [ ]:
dic_of_json = {}
for j in range(2007, 2022):
    if j == 2019 or j == 2011:
        continue
    with open(f'theses_{j}/theses_{j}.txt', 'r', encoding='utf-8') as f:  # parsing texts
        text_theses = f.read()
        texts = re.split(r'\n(Литература|Библиография)', text_theses)  # split by the Bibliography
        for index, text in enumerate(texts):
            if 'Bibliography' in text:  # if in English - skip
                continue
            if re.search(r'\n[А-ЯЁ]\.? [А-ЯЁ]\.? [^\s]*', text):  # searching for names
                start = re.search(r'\n[А-ЯЁ]\.? [А-ЯЁ]\.? [^\s]*', text).span()[0]  # name as a start-point
                text = text[start:].strip().split('\nСокращения')[0].split('\nСписок условных сокращений')[0].split('\nГлоссы')[0].split('\nСписок глосс')[0]  # cleaning from abbreviations
                text = re.sub(r"\n\(\d+[a-eа-г]?'*\)[^‘]+‘[^’]+?’\.?", '\n', text)  # glosses-deletion
                text = re.sub(r"\n\(\d+'*\)[^.]+?\.?", '\n', text)  # glosses-deletion
                text = re.sub(r'\n\D+(\d{4}) — (.|\n)+?\d\)?\.\n', '', text)  # deletion of bibliography residuals
                text = re.sub(r'\n[a-e]\.[^‘]+?‘[^’]+?’', '', text)  # glosses-deletion
                text_parts = text.split('\n')
                if len(text_parts) > 2 and not text_parts[2].isupper():  # finding name, title and affiliation (sorry for typos...)
                    for ind in range(2, len(text_parts)):
                        if text_parts[ind] and text_parts[ind].isupper() and ',' in text_parts[ind - 1] or 'Москва' in text_parts[ind - 1] or 'Санкт-Петербург' in text_parts[ind - 1]:
                            text_parts = text_parts[ind - 2:]
                            splitted_affilation = text_parts[ind - 1].strip().split(' ')
                            if len(splitted_affilation[-1]) > 2 and not (splitted_affilation[-1][0].isupper() and splitted_affilation[-1][1].islower()):
                                for s_af in range(len(splitted_affilation) - 1, -1, -1):
                                    if len(splitted_affilation[s_af]) > 2 and splitted_affilation[s_af][0].isupper() and splitted_affilation[s_af][1].islower():
                                        affilation = ' '.join(splitted_affilation[:s_af + 1])
                            else:
                                affilation = text_parts[ind - 1].strip()
                            name = text_parts[ind - 2].strip()
                            break
                else:
                    affilation = text_parts[1].strip()
                    name = text_parts[0].strip()
                ind = 2
                title = ''
                if text_parts and len(text_parts) >= 4:
                    while text_parts[ind].isupper():
                        title += text_parts[ind].strip() + ' '
                        ind += 1
                    string_words = text_parts[ind].split(' ')
                    for word in string_words:
                        if word.isupper():
                            title += word + ' '
                            rest = ''
                        else:
                            for letter in range(len(word)):
                                if letter < len(word) - 3 and word[letter].isupper() and word[letter + 1].isupper() and word[letter + 2].isupper():
                                    title += word[letter]
                                elif letter < len(word) - 3 and word[letter].isupper() and word[letter + 1].isupper() and not word[letter + 2].isupper():
                                    title += word[letter]
                                    break
                            rest = word[letter + 1:]
                            break
                    text = rest + ' '.join(text_parts[ind:])
                    # writing in dictionary what's left) 
                    dic_of_json[f'{index}_{j}'] = {'year': j, 'author': name.strip(), 'title': title.strip(), 'affilation': affilation.strip(), 'text': text.strip()}
clear_dic_of_json = dict()
for item in dic_of_json:  # cleaning the dictionary from residuals
    if dic_of_json[item]['title'] and dic_of_json[item]['author'] and dic_of_json[item]['affilation']:
        if '(' in dic_of_json[item]['author']:
            duplicate = dic_of_json[item]['author'].split('(')
            dic_of_json[item]['author'] = duplicate[0]
            dic_of_json[item]['title'] = dic_of_json[item]['affilation'] + dic_of_json[item]['title']
            dic_of_json[item]['affilation'] = duplicate[1].strip(')')
        if not dic_of_json[item]['title'].isupper() or not dic_of_json[item]['author'][0].isupper() or not dic_of_json[item]['affilation'][0].isupper():
            continue
        clear_dic_of_json[item] = dic_of_json[item]
with open('theses.json', 'w', encoding='utf-8') as json_file:  # put dictionary in JSON-file
    json.dump(clear_dic_of_json, json_file, indent=4, ensure_ascii=False)